<a href="https://colab.research.google.com/github/tcole333/data-centric-ai/blob/tcole/data_centric_ai_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import requests
import tarfile
import io
import numpy as np
from PIL import Image, ImageTk
from matplotlib import pyplot as plt
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.python.keras.preprocessing import dataset_utils
import os
import tensorflow as tf
from tensorflow import keras
import json
import sys
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [69]:
#clear directories so you have a fresh start
!rm -r '/content/data_sorted'
!rm -r '/content/data_preprocessed'

In [70]:
#download the files from github and extract them and move the sample labels to a new folder called test
!wget https://github.com/tcole333/data-centric-ai/blob/tcole/data_sorted.zip?raw=true
#tar = tarfile.open('data.tar.gz?raw=true')
#tar.extractall()
!unzip data_sorted.zip?raw=true
!rm -r "/content/data_sorted/train/junk_vals"
!wget https://github.com/tcole333/data-centric-ai/blob/main/label_book.tar.gz?raw=true
tar = tarfile.open('label_book.tar.gz?raw=true')
tar.extractall()
#set up the folder structure for the preprocessed data
!mv "/content/label_book" "/content/data_sorted/test"
!mkdir '/content/data_preprocessed'
!mkdir '/content/data_preprocessed/train'
!mkdir '/content/data_preprocessed/val'
!for num in i ii iii iv v vi vii viii ix x; do mkdir /content/data_preprocessed/train/$num ; mkdir /content/data_preprocessed/val/$num; done
!cp -r '/content/data_sorted/test' '/content/data_preprocessed/test'

--2021-06-24 18:58:18--  https://github.com/tcole333/data-centric-ai/blob/tcole/data_sorted.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tcole333/data-centric-ai/raw/tcole/data_sorted.zip [following]
--2021-06-24 18:58:18--  https://github.com/tcole333/data-centric-ai/raw/tcole/data_sorted.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tcole333/data-centric-ai/tcole/data_sorted.zip [following]
--2021-06-24 18:58:18--  https://raw.githubusercontent.com/tcole333/data-centric-ai/tcole/data_sorted.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP re

In [71]:
train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rescale=1/255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=False)

valid_datagen = ImageDataGenerator(rescale=1/255)

In [76]:
 batch_size = len(os.listdir('/content/data_preprocessed/train/ii'))
 batch_size

334

In [75]:
class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"]

for class_name in class_names:
  train_generator = train_datagen.flow_from_directory(
    '/content/data_sorted/train',
    target_size=(32,32),
    batch_size=500,
    classes=[class_name],
    save_to_dir='/content/data_preprocessed/train/'+class_name,
    save_prefix='aug',
    shuffle=True
  )
  batch = next(train_generator)
 
for class_name in class_names:
  batch_size = len([f for f in os.listdir('/content/data_sorted/val/'+ class_name) if os.path.isfile(os.path.join('/content/data_sorted/val/' + class_name, f))])
  validation_generator = valid_datagen.flow_from_directory(
        '/content/data_sorted/val',
        target_size=(32, 32),
        class_mode='categorical',
        classes=[class_name],
        batch_size=batch_size,
        shuffle=False,
        save_to_dir='/content/data_preprocessed/val/'+class_name)  
  batch = next(validation_generator)

Found 259 images belonging to 1 classes.
Found 167 images belonging to 1 classes.
Found 186 images belonging to 1 classes.
Found 278 images belonging to 1 classes.
Found 185 images belonging to 1 classes.
Found 162 images belonging to 1 classes.
Found 181 images belonging to 1 classes.
Found 178 images belonging to 1 classes.
Found 216 images belonging to 1 classes.
Found 188 images belonging to 1 classes.
Found 82 images belonging to 1 classes.
Found 82 images belonging to 1 classes.
Found 79 images belonging to 1 classes.
Found 84 images belonging to 1 classes.
Found 82 images belonging to 1 classes.
Found 82 images belonging to 1 classes.
Found 77 images belonging to 1 classes.
Found 83 images belonging to 1 classes.
Found 81 images belonging to 1 classes.
Found 81 images belonging to 1 classes.


In [77]:
directory = "/content/data_preprocessed"
user_data = directory + "/train"
valid_data = directory + "/val"
test_data = directory + "/test" # this can be the label book, or any other test set you create

### DO NOT MODIFY BELOW THIS LINE, THIS IS THE FIXED MODEL ###
batch_size = 8
tf.random.set_seed(123)


if __name__ == "__main__":
    train = tf.keras.preprocessing.image_dataset_from_directory(
        user_data,# + '/train',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    valid = tf.keras.preprocessing.image_dataset_from_directory(
        valid_data,# + '/val',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    total_length = ((train.cardinality() + valid.cardinality()) * batch_size).numpy()
    if total_length > 10_000:
        print(f"Dataset size larger than 10,000. Got {total_length} examples")
        sys.exit()

    test = tf.keras.preprocessing.image_dataset_from_directory(
        test_data,
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=False,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    base_model = tf.keras.applications.ResNet50(
        input_shape=(32, 32, 3),
        include_top=False,
        weights=None,
    )
    base_model = tf.keras.Model(
        base_model.inputs, outputs=[base_model.get_layer("conv2_block3_out").output]
    )

    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(10)(x)
    model = tf.keras.Model(inputs, x)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    model.summary()
    loss_0, acc_0 = model.evaluate(valid)
    print(f"loss {loss_0}, acc {acc_0}")

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        "best_model",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train,
        validation_data=valid,
        epochs=100,
        callbacks=[checkpoint],
    )

    model.load_weights("best_model")

    loss, acc = model.evaluate(valid)
    print(f"final loss {loss}, final acc {acc}")

    test_loss, test_acc = model.evaluate(test)
    print(f"test loss {test_loss}, test acc {test_acc}")

   

Found 4000 files belonging to 10 classes.
Found 1626 files belonging to 10 classes.
Found 52 files belonging to 10 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
tf.__operators__.getitem_4 ( (None, 32, 32, 3)         0         
_________________________________________________________________
tf.nn.bias_add_4 (TFOpLambda (None, 32, 32, 3)         0         
_________________________________________________________________
model_8 (Functional)         (None, 8, 8, 256)         229760    
_________________________________________________________________
global_average_pooling2d_4 ( (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 232,330
Trainable params: 229,386
Non-trainable params: 2,944
_________________________________________________

In [78]:
#view predictions
pred_classes = model.predict(test).argmax(axis=-1) + 1

pred_classes

array([ 3,  1,  1,  1,  1,  2,  7,  3,  2,  2,  3,  3,  1,  4,  3,  7,  3,
        4,  7,  2,  3,  1,  9,  4,  3,  9,  2,  5,  1,  5,  5,  5,  1,  6,
        3,  6,  6,  7,  3,  7,  7,  3,  8,  8,  8,  3,  8, 10, 10,  7,  1,
        3])